检查各个变量的方差是否太小，太平稳

In [31]:
import os
import numpy as np
import pandas as pd

# 定义要筛选的字符串和文件夹路径
target_str = 'data'
target_str_interv = 'data_interv'
folder_path = '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/' # downstream_less, upstream_less3

# 获取文件夹中所有文件名中含有 target_str 或 target_str_interv 的 npy 文件列表，并按文件名排序
file_list = sorted([os.path.join(folder_path, file) for file in os.listdir(folder_path) if (target_str in file or target_str_interv in file) and file.endswith('.npy')])
print('符合条件的文件列表：', file_list)

# 计算每个文件为 numpy 数组，并计算每列的方差
variances = {}
for file in file_list:
    arr = np.load(file)
    var = np.var(arr, axis=0)
    variances[os.path.basename(file)] = var

# 将方差结果转化为 dataframe，并计算含“data_interv”文件的均值和其他文件的均值
var_df = pd.DataFrame(variances)
var_df_interv = var_df.filter(regex=target_str_interv)
var_mean_interv = var_df_interv.mean(axis=1)
var_df_others = var_df.drop(columns=var_df_interv.columns)
var_mean_others = var_df_others.mean(axis=1)

# 将每个文件的方差结果保存为一个 dataframe，并保留小数点后5位
var_df_indiv = var_df.T.reset_index()
var_df_indiv = var_df_indiv.round(5)
var_df_mean = pd.DataFrame({'var_mean_interv': var_mean_interv, 'var_mean_others': var_mean_others}).round(5).T.reset_index()

# 将两个结果合并为一个 dataframe，并保存为 csv 文件
result = pd.concat([var_df_indiv, var_df_mean], axis=0)
result.rename(columns={'index': 'filename'}, inplace=True)
result.to_csv(folder_path+'variances.csv', index=False)
print('结果已保存为 variances.csv 文件')


符合条件的文件列表： ['/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data1.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data2.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data3.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data4.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data5.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data6.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data7.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data_interv1.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data_interv2.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data_interv3.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data_interv4.npy', '/home/lipeiwen.lpw/realdata_preprocess/train_data/upstream_less3/data_interv5.npy', '/home/lipeiwen.l

In [32]:
# 统计小于 0.005 的个数
print(f'var_mean_interv 中小于 0.003 的个数为：{(var_mean_interv < 0.003).sum()}')
print(f'var_mean_others 中小于 0.003 的个数为：{(var_mean_others < 0.003).sum()}')

var_mean_interv 中小于 0.003 的个数为：0
var_mean_others 中小于 0.003 的个数为：28
